In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential, load_model, Model
from keras.layers import Input, average, concatenate, GlobalAveragePooling2D
from keras.layers import TimeDistributed, GlobalAveragePooling1D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential, load_model, Model
from pathlib import Path
import os

from keras.layers.core import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Setup
path = './data/hmdb51'
path_rowframes = './data/hmdb51/rawframes/'
path_annotations = './data/hmdb51/annotations/'

# Parametri Comuni
img_height = 224
img_width = 224
image_shape=(img_height, img_width)
batch_size = 128
num_classes = 51

# Parametri del temporal batch generator
num_of_snip=1
opt_flow_len=10

# Parametri di evaluation
fuse_method = 'average'

In [40]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.mobilenet import  MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Flatten


def create_model(input = (256,256,3)):
    model = Sequential()
    #
    model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = input))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    #
    model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    # fully connected
#     model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    return model

In [41]:
def two_stream_fuse(spatial_model, temporal_model):
    # spatial stream (frozen)
    cnn_spatial = spatial_model

    # temporal stream (frozen)
    cnn_temporal = temporal_model

    # fused by taking average
#     outputs = average([cnn_spatial.output, cnn_temporal.output])
    x = Concatenate()([cnn_spatial, cnn_temporal])
    x = Flatten()(x)
    x = Dropout(0.25)(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(5, activation='softmax')(x)

    model = Model([cnn_spatial.input, temporal_model.input], outputs)
#     model = Model(Concatenate(axis = -1)([cnn_spatial, cnn_temporal]))
    return model
    

In [42]:
model = two_stream_fuse(create_model(), create_model())
optimizer = Adam()
model.compile(loss=keras.losses.sparse_categorical_crossentropy, 
                   metrics=['sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'], 
                   optimizer=optimizer)

TypeError: object of type 'NoneType' has no len()

In [32]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d_40_input (InputLayer)   [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_42_input (InputLayer)   [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_40 (Conv2D)             (None, 256, 256, 8)  608         ['conv2d_40_input[0][0]']        
                                                                                            